In [1]:
# For gdrive mounting
# from google.colab import drive
# drive.mount('/content/drive')
# wd_url = '/content/drive/MyDrive/project_files/' 
# import sys
# sys.path.append(wd_url)

# For public repo
#!git clone https://github.com/Juniper82/SIDIS-Affinity

# ! git pull
import numpy as np
import pandas as pd
from pandas import read_excel 
from copy import deepcopy
from ipywidgets import *
import logging, os 
logging.disable(logging.WARNING) 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
# print(tf.__version__)

#%cd '/content/SIDIS-Affinity'
from plottools import plotEIC1

#!apt install texlive-fonts-recommended #texlive-fonts-extra 
%matplotlib inline

In [2]:
fname = './expdata/eic.xlsx'
data=pd.read_excel(fname,index_col=0,engine='openpyxl')

data.pT.max()
data.keys()

test_features = deepcopy(data)
test_features = test_features.drop(columns=['W2', 'hadron', 'target'])

In [3]:
tmd_model_name = 'tmd_DNN'
tmd_model = tf.keras.models.load_model('./models/'+tmd_model_name)
target_model_name = 'target_DNN'
target_model = tf.keras.models.load_model('./models/'+target_model_name)
collinear_model_name = 'collinear_DNN'
collinear_model = tf.keras.models.load_model('./models/'+collinear_model_name)
current_model_name = 'current_DNN'
current_model = tf.keras.models.load_model('./models/'+current_model_name)

In [4]:
# Create a slider widget for interactive Rmax selection
R0maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R1maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R2maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
regionDrop=widgets.Dropdown(
    options=[('tmd', 'tmdaff'), ('target', 'targetaff'),('collinear','collinearaff'),('current','currentaff')],
    value='tmdaff',
    description='Region:')

# Define a function to predict tmdaff that corresponds the widgets Rmax values and return plot
def plot(R0max=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop):
       
    test_features['R0max'] = np.ones(len(test_features))*R0maxSlider.value
    test_features['R1max'] = np.ones(len(test_features))*R1maxSlider.value
    test_features['R2max'] = np.ones(len(test_features))*R2maxSlider.value

    if region == 'tmdaff':
        data[region] = tmd_model.predict(test_features).flatten()
        neg = data.query('tmdaff < 0')
        print(neg)
    elif region == 'targetaff':
        data[region] = target_model.predict(test_features).flatten()
        neg = data.query('targetaff < 0')
        print(neg)
    elif region == 'collinearaff':
        data[region] = collinear_model.predict(test_features).flatten()
        neg = data.query('collinearaff < 0')
        print(neg)
    else:
        data[region] = current_model.predict(test_features).flatten()
        neg = data.query('currentaff < 0')
        print(neg)
    print(min(data[region]))
    # Structural pattern matching coming in python 3.10.0 final to be released this October (approx. 10/4/21)
    # match region:
    #   case "tmdaff":
    #     data[region] = tmd_model.predict(test_features).flatten()
    #   case "targetaff":
    #     data[region] = target_model.predict(test_features).flatten()
    #   case "collinearaff":
    #     data[region] = collinear_model.predict(test_features).flatten()
    #   case "cuurentaff":
    #     data[region] = current_model.predict(test_features).flatten()


    return plotEIC1(data, hadron = 'pi+', affinity = region, plotx = 'pT', ploty = 'z', cmap_name = 'Spectral', yscale = 'linear')

In [5]:
#Initiate user interaction 
execute = interactive(plot, {'manual': True},R0=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop)
execute

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='R0max', max=1.0, min=0.1, s…